### Лабораторная работа 4

#### Решение систем линейных алгебраических уравнений прямыми методами. Теория возмущений

Прокопьев Григорий <br>
Вариант 21 (3.1.21, 3.5.6, 3.6.5)

In [1]:
import numpy as np
import numpy.linalg as nla

In [2]:
def pretty_print(A, b, w=10):
    for i in range(n):
        print('|', end='')
        
        for j in range(n):
            print('{:f}'.format(A[i, j]).rjust(w), end='')
            
        print(' |\t', '| x_{:d} |'.format(i), end='')
        
        if i == n//2:
            print('   =', '|{:.0f}|'.format(b[i]).rjust(w-3))
        else:
            print('|{:.0f}|'.format(b[i]).rjust(w+2))

#### Задача 3.1.21:

Дана система уравнений $Ax=b$ порядка $n$. Исследовать зависимость погрешности решения $x$ от погрешности правой части системы $b$.

1) Задание системы:

In [3]:
n = 6
N = 21

In [4]:
A = np.empty((n, n))
b = np.empty(n)

In [5]:
for i in range(n):
    for j in range(n):
        c = 0.1 * N * (i+1) * (j+1)
        A[i,j] = 1000 / ((3+c) * c**2)
b.fill(N)

pretty_print(A, b)

| 44.462229  7.873520  2.709168  1.243187  0.671874  0.403770 |	 | x_0 |        |21|
|  7.873520  1.243187  0.403770  0.178944  0.094482  0.055841 |	 | x_1 |        |21|
|  2.709168  0.403770  0.127830  0.055841  0.029212  0.017154 |	 | x_2 |        |21|
|  1.243187  0.178944  0.055841  0.024201  0.012598  0.007372 |	 | x_3 |   =    |21|
|  0.671874  0.094482  0.029212  0.012598  0.006537  0.003817 |	 | x_4 |        |21|
|  0.403770  0.055841  0.017154  0.007372  0.003817  0.002226 |	 | x_5 |        |21|


Слева выведена матрица $ А $, справа $-$ правая часть системы $b$

Найдем решение системы с помощью встроенной функции Numpy:

In [6]:
x = nla.solve(A, b)

for i in range(n):
    print('{:10.6f}'.format(x[i]).rjust(20))

       583328.374632
   -128347861.371134
   2878120492.182710
 -17178467334.355768
  35516027740.729752
 -23079784843.863583


2) Найдем число обусловленности матрицы $A$:

In [7]:
cond = nla.cond(A, p=np.inf)
cond

3749056998771.7

3) Внесем погрешность по очереди в компоненты вектора $b$ и найдем относительные погрешности решений полученных систем:

In [8]:
d = np.empty(n)
b_err = np.empty(n)

for i in range(n):
    b_ = b.copy()
    # за погрешность равна 0.1 от значения b[i]
    b_[i] *= 1.1
    
    x_ = nla.solve(A, b_)
    
    d[i] = nla.norm(x - x_, ord=np.inf)
    b_err[i] = nla.norm(b - b_, ord=np.inf)
    
d /= nla.norm(x, ord=np.inf)
b_err /= nla.norm(b, ord=np.inf)

4) Определим компоненту $b$, которая оказывает наибольшее влияние на погрешность решения:

In [9]:
d

array([2.55215390e-05, 6.00894225e-03, 1.38259580e-01, 8.36475577e-01,
       1.74390313e+00, 1.13970371e+00])

In [10]:
np.argmax(d)

4

5) Оценим теоретически относительную погрешность и сравним с практически посчитанной погрешностью:

$\delta(x^{m}) \leq cond(A) \cdot \delta(b^{m})$

$cond(A) \cdot \delta(b^{m})$:

In [11]:
cond * b_err[4]

374905699877.1702

Практически посчитанная погрешность:

In [12]:
d[4]

1.743903131662153

Как видно, теоретическая верхняя граница относительной погрешности больше практической погрешности: это числа разных порядков. Это объясняется плохой обусловленностью матрицы $A$: ее число обусловленности $cond(A)$ гораздо больше 1:

In [13]:
cond

3749056998771.7

#### Задача 3.5.6

Дана система уравнений $Ax=b$ порядка $n$ с симметричной положительно определенной матрицей $A$. Решить систему  методом Холецкого.

0) Задание системы:

In [14]:
n = 25
m = 10

In [15]:
A = np.empty((n,n))
b = np.empty(n)

Элементы матрицы $A$ вычисляются по формуле:

\begin{equation}
A_{ij} = 
 \begin{cases}
   \frac{i+j}{m+n}, &i \neq j\\
   n+m^{2}+\frac{j}{m}+\frac{i}{n}, &i = j
 \end{cases}
\end{equation}

$$ i, j = 1,\dots,n $$

А элементы вектора $b$ $-$ по формуле:

$$ b_{i} = i^{2} - n;\quad\\
i = 1,\dots,n$$

In [16]:
for i in range(n):
    for j in range(n):
        if i != j:
            A[i,j] = (i+j+2)/(n+m)
        else:
            A[i,j] = n + m**2 + (j+1)/m + (i+1)/n
            
    b[i] = (i+1)**2 - n

1) Найдем разложение Холецкого матрицы $A$, используя встроенную функцию numpy:

In [17]:
L = nla.cholesky(A)

2) Решим последовательно системы $Ly=b$ и $L^{T}x=y$

In [18]:
y = nla.solve(L, b)

In [19]:
x = nla.solve(L.T, y)

Sanity check:

In [20]:
np.allclose(x, nla.solve(A, b))

True

#### Задание 3.6.5

Дана система уравнений $Ax=b$ порядка $n$, где $A=A(t)$, $t$ - параметр. Исследовать  зависимость решения системы $Ax=b$ от вычислительной погрешности при заданных значениях параметра t. 

Формула для задания элементов матрицы $A$:

\begin{equation}
A_{ij} = 
 \begin{cases}
    q_{M}^{j}, &i \neq j\\
    q_{M}^{j}+t, &i = j
 \end{cases}
\end{equation}

$$ 
i,j = 1,\dots, n;\quad\\
q_{M} = 0.993 + (-1)^{M}\cdot M \cdot 10^{-4};\quad\\
t = 0.0001, 1, 10000
$$

Формула для задания элементов вектора $b$:

$$
b_{j} = q_{M}^{(n+1-j)}, \\
j = 1, \dots, n
$$

$$M = 5,\quad\\
n = 45,\quad\\
m = 4\quad$$

1) Функция, реализующая метод Гаусса:

In [21]:
def Gauss_method(A, b, fround):
    # fround - функция округления до m знаков после запятой
    # для вычисления без округления можно задать fround=lambda x: x 
    A_ = A.copy()
    b_ = b.copy()
    
    for j in range(A.shape[1]):
        r = np.argmax(np.abs(A_[j:,j])) + j
        if r != j:
            A_[[r, j]] = A_[[j, r]]
            b_[[r, j]] = b_[[j, r]]
        
        div = A_[j,j]
        A_[j] = fround(A_[j] / div)
        b_[j] = fround(b_[j] / div)
        
        
        for i in range(j+1, A.shape[0]):
            w = -A_[i, j]
            A_[i] = fround(A_[i] + w * A_[j])
            b_[i] = fround(b_[i] + w * b_[j])
            
        x = np.zeros_like(b)
        for i in range(A.shape[1]-1, -1, -1):
            x[i] = fround(b_[i] - A_[i,i+1:] @ x[i+1:])
            
    return x

2) Задание системы при разных параметрах $ t $ и решение системы и системы после округления элементов:

In [22]:
M = 5
n = 45
m = 4
tt = [1e-4, 1, 1e4]

In [23]:
A = np.empty((n, n))
b = np.empty(n)
d = dict()

for t in tt:
    q = 0.993 + (-1)**M * M * 1e-4
    
    A = np.tile([q**j for j in range(1, n+1)], (n,1))
    A += np.diag(np.diag(A) + t)
    b = np.array([q**(n + 1 - j) for j in range(1, n+1)])
    x = Gauss_method(A, b, fround=lambda x: x)
    
    A1 = np.round(A, m)
    b1 = np.round(b, m)
    x1 = Gauss_method(A1, b1, fround=lambda x: np.round(x, m))
    
    # считаем относительную ошибку по inf-норме аналогично первому номеру
    d[t] = nla.norm(x - x1, np.inf)/nla.norm(x, np.inf)

Выведем значения относительных ошибок:

In [24]:
for k, v in sorted(d.items(), key=lambda x: x[1]):
    print("t = ", k, "; error = {:f}".format(v), sep='')

t = 1; error = 0.001115
t = 0.0001; error = 0.002105
t = 10000.0; error = 0.293751


- Ошибка при прибавлении целого $ t = 1 $ к диагональным элементам матрицы $ A $: наименьшая из всех. 
- Ошибка при прибавлении малого $ t = 0.0001 $ больше предыдущей: это объясняется тем, что, при возведении в степень элементов с погрешностями (при формировании матрицы $ A $), у этих элементов образуются "хвосты", которые потом теряются при округлении до $ m $ знаков после запятой.
- Ошибка при прибавлении наибольшего  $ t = 10000 $: ошибка наибольшая - ожидаемо